In [19]:
pip install streamlit

Note: you may need to restart the kernel to use updated packages.


In [23]:
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
import streamlit as st
import matplotlib.pyplot as plt

# Fetch stock data
def fetch_stock_data(ticker, period='1y', interval='1d'):
    data = yf.download(ticker, period=period, interval=interval)
    data['Return'] = data['Close'].pct_change()  # Percentage change
    data['Price_Change'] = data['Close'].diff()  # Absolute price change
    data['Target'] = (data['Return'] > 0).astype(int)  # Classification target
    data.dropna(inplace=True)
    return data

# Add features
def add_features(data):
    data['SMA_10'] = data['Close'].rolling(window=10).mean()
    data['SMA_50'] = data['Close'].rolling(window=50).mean()
    data['RSI'] = compute_rsi(data['Close'], window=14)
    data.dropna(inplace=True)
    return data

# Compute RSI
def compute_rsi(series, window=14):
    delta = series.diff(1)
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

# Train models
def train_models(data):
    features = data[['SMA_10', 'SMA_50', 'RSI']]
    classification_target = data['Target']
    regression_target = data['Price_Change']
    
    clf_model = RandomForestClassifier(n_estimators=100, random_state=42)
    clf_model.fit(features, classification_target)
    
    reg_model = LinearRegression()
    reg_model.fit(features, regression_target)
    
    return clf_model, reg_model

# Live prediction
def live_prediction(ticker, clf_model, reg_model):
    data = fetch_stock_data(ticker, period='5d', interval='15m')
    data = add_features(data)
    
    latest_features = data[['SMA_10', 'SMA_50', 'RSI']].iloc[-1:].values
    
    rise_fall_prediction = clf_model.predict(latest_features)
    rise_fall_result = "Price will Rise" if rise_fall_prediction[0] == 1 else "Price will Fall"
    
    price_change_prediction = reg_model.predict(latest_features)[0]
    price_change_result = f"The price will change by approximately ${price_change_prediction:.2f}."
    
    return f"{rise_fall_result}\n{price_change_result}"

# Streamlit UI
st.title("Stock Price Prediction App")
st.sidebar.header("User Input")
ticker = st.sidebar.text_input("Enter Stock Ticker", "AAPL")
period = st.sidebar.selectbox("Select Period", ["1mo", "3mo", "6mo", "1y"])
interval = st.sidebar.selectbox("Select Interval", ["1d", "1h", "15m"])

if st.button("Predict"):
    with st.spinner("Fetching data and predicting..."):
        data = fetch_stock_data(ticker, period, interval)
        data = add_features(data)
        clf_model, reg_model = train_models(data)
        prediction = live_prediction(ticker, clf_model, reg_model)
        
        st.success(prediction)
        
        st.subheader("Stock Prices")
        plt.figure(figsize=(10, 5))
        plt.plot(data['Close'], label="Close Price")
        plt.title(f"{ticker} Stock Prices")
        plt.xlabel("Date")
        plt.ylabel("Price")
        plt.legend()
        st.pyplot(plt)
